In [75]:
from tfga import GeometricAlgebra
import tensorflow as tf

# Quantum Electrodynamics with Geometric Algebra

## Theory overview
Quantum Electrodynamics (QED) describes electrons, positrons (anti-electrons) and photons in a 4-dimensional spacetime with fields defined for all spacetime positions $X$. The 4-dimensional spacetime can be described by the [Spacetime Algebra (STA)](https://en.wikipedia.org/wiki/Spacetime_algebra) with basis vectors $\gamma_0, \gamma_1, \gamma_2, \gamma_3$ and corresponding metric $[1, -1, -1, -1]$. It contains two fields. The electron-positron field is a bispinor-field $\psi(X)$ which in the context of Geometric Algebra (GA) is described by odd-grade multivectors of the STA. The photon field $A(X)$ is a vector-field (ie. multivectors of degree 1, one for each dimension).

A field configuration, also known as a path, $P(X)$ contains values for the two fields at every spacetime position. Our goal is to calculate the QED action using GA which allows us to use algorithms that solve for field configurations . The action is the negative log-likelihood (NLL) of the field configuration, meaning it is a number which tells how likely a given field configuration is. It is not a probability as it is unnormalized. However even with only the NLL we can use sampling algorithms (eg. [Markov-Chain Monte-Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo), [Variational Inference](https://en.wikipedia.org/wiki/Variational_Bayesian_methods)) to sample field configurations so that the sampled distribution matches the normalized distribution.

The Lagrangian is given in Hestenes' article [Real Dirac Theory](https://www.researchgate.net/publication/266578033_REAL_DIRAC_THEORY) in equation (B.6) as

$\mathcal{L} = \langle \hbar (\nabla \psi(X)) i \gamma_3 \widetilde{\psi}(X) - e A(X) \psi(X) \gamma_0 \widetilde{\psi}(X) - m \psi(X) \widetilde{\psi}(X) \rangle$

where $\langle ... \rangle$ denotes getting the scalar part, $i = \gamma_2 \gamma_1$, $\nabla = \sum_{i=0}^{3} \gamma_i \delta^i$ and $\widetilde{\psi}(X)$ is the grade-reversal of $\psi$.

The action $S(P)$ for a field-configuration $P=(\psi, A)$ is calculated by integrating the Lagrangian $\mathcal{L}(P, X)$ over all space-time positions $X$.

$S(\psi, A) = \int_{X \in \mathcal{X}} \mathcal{L}(\psi, A, X) dx$

Finally as we are doing this numerically we need to discretize spacetime into a 4-dimensional grid. Integrals over spacetime then become sums over the grid. Derivatives become finite-differences or more complicated operations to avoid the [aliasing](https://arxiv.org/abs/hep-lat/0207008) which results in the [fermion doubling](https://en.wikipedia.org/wiki/Fermion_doubling) problem.

## Getting started
Let's start by defining the spacetime algebra as a geometric algebra in 1 time and 3 space dimensions with metric $[1, -1, -1, -1]$.

In [76]:
sta_ga = GeometricAlgebra([1, -1, -1, -1])
print(sta_ga.basis_mvs)

[MultiVector[1.00*e_0], MultiVector[1.00*e_1], MultiVector[1.00*e_2], MultiVector[1.00*e_3]]


We can see our four basis vectors displayed here each with a different ... basis. Let's try squaring them.

In [77]:
print("e_0^2:", sta_ga.basis_mvs[0] * sta_ga.basis_mvs[0])
print("e_1^2:", sta_ga.basis_mvs[1] * sta_ga.basis_mvs[1])
print("e_2^2:", sta_ga.basis_mvs[2] * sta_ga.basis_mvs[2])
print("e_3^2:", sta_ga.basis_mvs[3] * sta_ga.basis_mvs[3])

e_0^2: MultiVector[1.00*1]
e_1^2: MultiVector[-1.00*1]
e_2^2: MultiVector[-1.00*1]
e_3^2: MultiVector[-1.00*1]


Squaring the basis vectors gave us back another purely scalar multivector. The squared bases indeed return the correct metric.

We can create new multivectors of different kinds using the geometric algebra `sta_ga` object. Let's create some vectors such as the elements of the photon field and perform some operations to get a feel for them. We can use the methods on `sta_ga`, most of which take a `batch_shape` that says how many elements you want (`[]` meaning just a single one) and a `kind` that describes which elements it will set (eg. `"even"`, `"mv"` (meaning all), `"vector"`, `"scalar"`, ...). Alternatively we can just build everything out of the basis vectors ourselves by adding and multiplying them.

In [78]:
v1 = sta_ga.ones(batch_shape=[], kind="vector")
print("v1:", v1)

v2 = sta_ga.basis_mvs[0] + sta_ga.basis_mvs[1]
print("v2:", v2)

print("v1 * v2 (Geometric product):", v1 * v2)
print("v1 | v2 (Inner product):", v1 | v2)
print("v1 ^ v2 (Exterior product):", v1 ^ v2)

v3 = v1 + v2
print("v3 = v1 + v2:", v3)
print("v1 ^ v3:", v1 | v3)
print("v1 | v3:", v1 | v3)

v4 = v1 * v2
print("v4 = v1 * v2:", v3)
print("v1^-1 * v4:", v1.inverse() * v4, "should be", v2)

v1: MultiVector[1.00*e_0 + 1.00*e_1 + 1.00*e_2 + 1.00*e_3]
v2: MultiVector[1.00*e_0 + 1.00*e_1]
v1 * v2 (Geometric product): MultiVector[-1.00*e_02 + -1.00*e_12 + -1.00*e_03 + -1.00*e_13]
v1 | v2 (Inner product): MultiVector[]
v1 ^ v2 (Exterior product): MultiVector[-1.00*e_02 + -1.00*e_12 + -1.00*e_03 + -1.00*e_13]
v3 = v1 + v2: MultiVector[2.00*e_0 + 2.00*e_1 + 1.00*e_2 + 1.00*e_3]
v1 ^ v3: MultiVector[-2.00*1]
v1 | v3: MultiVector[-2.00*1]
v4 = v1 * v2: MultiVector[2.00*e_0 + 2.00*e_1 + 1.00*e_2 + 1.00*e_3]
v1^-1 * v4: MultiVector[1.00*e_0 + 1.00*e_1] should be MultiVector[1.00*e_0 + 1.00*e_1]


Now let's do the same for the bispinors (elements of even degree).

In [79]:
b1 = sta_ga.ones(batch_shape=[], kind="even")
print("b1:", b1)

b2 = 4.0 + sta_ga.basis_mvs[0] * sta_ga.basis_mvs[1] + sta_ga.basis_mvs[0] * sta_ga.basis_mvs[1]
print("b2:", b2)

print("b1 | b2:", b1 | b2)
print("b1 ^ b2:", b1 ^ b2)

b3 = b1 * b2
print("b3 = b1 * b2:", b3)
print("b3 * b2^-1:", b3 * b2.inverse(), "should be", b1)

print("~b2 (Grade reversal):", ~b2)
print("Scalar part of b2:", b2[""])
print("e_01 part of b2:", b2["01"])

b1: MultiVector[1.00*1 + 1.00*e_01 + 1.00*e_02 + 1.00*e_03 + 1.00*e_12 + 1.00*e_13 + 1.00*e_23 + 1.00*e_0123]
b2: MultiVector[2.00*e_01 + 4.00*1]
b1 | b2: MultiVector[6.00*e_01 + 6.00*1 + 4.00*e_12 + 4.00*e_02 + 4.00*e_13 + 4.00*e_03 + 6.00*e_0123 + 6.00*e_23]
b1 ^ b2: MultiVector[2.00*e_12 + 2.00*e_02 + 2.00*e_13 + 2.00*e_03]
b3 = b1 * b2: MultiVector[6.00*e_01 + 6.00*1 + 6.00*e_12 + 6.00*e_02 + 6.00*e_13 + 6.00*e_03 + 6.00*e_0123 + 6.00*e_23]
b3 * b2^-1: MultiVector[1.00*1 + 1.00*e_01 + 1.00*e_02 + 1.00*e_12 + 1.00*e_03 + 1.00*e_13 + 1.00*e_23 + 1.00*e_0123] should be MultiVector[1.00*1 + 1.00*e_01 + 1.00*e_02 + 1.00*e_03 + 1.00*e_12 + 1.00*e_13 + 1.00*e_23 + 1.00*e_0123]
~b2 (Grade reversal): MultiVector[-2.00*e_01 + 4.00*1]
Scalar part of b2: MultiVector[4.00*1]
e_01 part of b2: MultiVector[2.00*e_01]


Now we hopefully have some feel for how to operate with the geometric algebra numbers. So far we only worked with single numbers, but we can define a field (ie. a number for every grid point) by passing in a `batch_shape` that is the size of our grid. When printing the fields we won't see the actual numbers anymore, we will only see which blades are non-zero and the batch shape. However we can still access all of the numbers with the usual indexing rules.

In [80]:
a = sta_ga.ones(batch_shape=[10, 10, 10, 10], kind="vector")
print("A(X):", a)

print("A(t=0, x=5, y=3, z=9):", a[0, 5, 3, 9])
print("A(t=0, z=[3,4,5]):", a[0, :, :, 3:6])
print("e_0 part of A(X):", a["0"])

print("A(0, 0, 0, 0) * ~A(0, 0, 0, 0):", (a * ~a)[0, 0, 0, 0])

A(X): MultiVector[batch_shape=(10, 10, 10, 10), blades=[e_0, e_1, e_2, e_3]]
A(t=0, x=5, y=3, z=9): MultiVector[1.00*e_0 + 1.00*e_1 + 1.00*e_2 + 1.00*e_3]
A(t=0, z=[3,4,5]): MultiVector[batch_shape=(10, 10, 3), blades=[e_0, e_1, e_2, e_3]]
e_0 part of A(X): MultiVector[batch_shape=(10, 10, 10, 10), blades=[e_0]]
A(0, 0, 0, 0) * ~A(0, 0, 0, 0): MultiVector[-2.00*1]


By now you will probably believe me that we can do make a bispinor field, so instead let's see how we can calculate derivatives.

As mentioned in the beginning, derivatives become finite differences. To calculate finite differences we can take a copy of the field, shift it back by one in a dimension and subtract it. For instance of we were to calculate the derivative
in the time direction we would shift the entire field by -1 along the time axis to get `A(X + TimeDirection * GridSpacing)` and subtract the actual field from this shifted field. All that is left then is to divide by the grid spacing.

`d/dt A(X) = (A(X + TimeDirection * GridSpacing) - A(X)) / GridSpacing`

To actually do the shifting we will use the `with_changes` method which allows copying of the multivector and overriding of its blade values so we will just shift the blade values themselves using [tf.roll](https://www.tensorflow.org/api_docs/python/tf/roll). A better abstraction that doesn't require using the interval blade values might be added later.

In [81]:
def finite_differences(field, axis, spacing):
    shifted_blade_values = tf.roll(field.blade_values, shift=-1, axis=axis)

    # Keep all properties of the old field, just use
    # the new shifted field values instead.
    shifted_field = field.with_changes(
        blade_values=shifted_blade_values
    )

    return (shifted_field - field) / spacing

deriv_t_a = finite_differences(a, axis=0, spacing=0.1)
print("d/dt A(X) = (A(X + TimeDirection * GridSpacing) - A(X)) / GridSpacing:", deriv_t_a)
print("d/dt A(0, 0, 0, 0):", deriv_t_a[0, 0, 0, 0])

d/dt A(X) = (A(X + TimeDirection * GridSpacing) - A(X)) / GridSpacing: MultiVector[batch_shape=(10, 10, 10, 10), blades=[e_0, e_1, e_2, e_3]]
d/dt A(0, 0, 0, 0): MultiVector[]


Maybe expectedly, as our field is just a constant value everywhere, we are left with a field that is zero everywhere. Now we have a finite differences operation that will work on fields of any kind.

Now we have all the tools we need to actually calculate the QED action given a field configuration. As a reminder, the QED Lagrangian is given by

$\mathcal{L} = \langle \hbar (\nabla \psi(X)) i \gamma_3 \widetilde{\psi}(X) - e A(X) \psi(X) \gamma_0 \widetilde{\psi}(X) - m \psi(X) \widetilde{\psi}(X) \rangle$

and the action $S(\psi, A)$ is the spacetime integral (now sum) over it.

Let's start with the mass term on the right $m \psi(X) \widetilde{\psi}(X)$.

In [82]:
def get_mass_term(psi, electron_mass):
    return electron_mass * psi * ~psi

# Define psi as some arbitrary even-graded field for now
psi = sta_ga.ones(batch_shape=[10, 10, 10, 10], kind="even") + sta_ga.ones(batch_shape=[10, 10, 10, 10], kind="scalar")
print("Psi:", psi)
print("Psi at (0, 0, 0, 0):", psi[0, 0, 0, 0])

# The electron mass in planck units (hbar=1, c=1) is actually not 1 but something tiny.
# However we won't bother with it for now.
mass_term = get_mass_term(psi=psi, electron_mass=1.0)
print("Mass term:", mass_term)
print("Mass term at (0, 0, 0, 0):", mass_term[0, 0, 0, 0])

Psi: MultiVector[batch_shape=(10, 10, 10, 10), blades=[1, e_01, e_02, e_03, e_12, e_13, e_23, e_0123]]
Psi at (0, 0, 0, 0): MultiVector[2.00*1 + 1.00*e_01 + 1.00*e_02 + 1.00*e_03 + 1.00*e_12 + 1.00*e_13 + 1.00*e_23 + 1.00*e_0123]
Mass term: MultiVector[batch_shape=(10, 10, 10, 10), blades=[1, e_01, e_02, e_03, e_12, e_13, e_23, e_0123]]
Mass term at (0, 0, 0, 0): MultiVector[3.00*1 + 2.00*e_0123]


Next the interaction term in the center that describes the scattering between the electron-positron field and the photon field $e A(X) \psi(X) \gamma_0 \widetilde{\psi}(X)$.

In [83]:
def get_interaction_term(psi, a, electron_charge):
    return electron_charge * a * psi * sta_ga.basis_mvs[0] * ~psi

interaction_term = get_interaction_term(psi=psi, a=a, electron_charge=1.0)
print("Interaction term:", interaction_term)
print("Interaction term at (0, 0, 0, 0):", interaction_term[0, 0, 0, 0])

Interaction term: MultiVector[batch_shape=(10, 10, 10, 10), blades=[1, e_01, e_02, e_03, e_12, e_13, e_23, e_0123]]
Interaction term at (0, 0, 0, 0): MultiVector[25.00*1 + -13.00*e_01 + -13.00*e_02 + -21.00*e_03 + -8.00*e_13 + -8.00*e_23]


And finally the momentum term for which we needed the finite differences $\hbar (\nabla \psi(X)) i \gamma_3 \widetilde{\psi}(X)$.

In [84]:
def get_momentum_term(psi, spacing, hbar):
    # Nabla Psi
    dt_psi = finite_differences(psi, axis=0, spacing=spacing)
    dx_psi = finite_differences(psi, axis=1, spacing=spacing)
    dy_psi = finite_differences(psi, axis=2, spacing=spacing)
    dz_psi = finite_differences(psi, axis=3, spacing=spacing)
    d_psi = dt_psi + dx_psi + dy_psi + dz_psi

    return hbar * d_psi * sta_ga.basis_mvs[1] * sta_ga.basis_mvs[2] * sta_ga.basis_mvs[3] * ~psi

momentum_term = get_momentum_term(psi=psi, spacing=0.1, hbar=1.0)

print("Momentum term:", momentum_term)
print("Momentum term at (0, 0, 0, 0):", momentum_term[0, 0, 0, 0]) # Still zero ;(

Momentum term: MultiVector[batch_shape=(10, 10, 10, 10), blades=[e_123, e_023, e_013, e_012, e_3, e_2, e_1, e_0]]
Momentum term at (0, 0, 0, 0): MultiVector[]


Now that we have all the terms, we can add them up, sum over all grid points and take the scalar part to get the action.

In [85]:
def get_action(psi, a, spacing, electron_mass, electron_charge, hbar):
    mass_term = get_mass_term(psi=psi, electron_mass=electron_mass)
    interaction_term = get_interaction_term(psi=psi, a=a, electron_charge=electron_charge)
    momentum_term = get_momentum_term(psi=psi, spacing=spacing, hbar=hbar)

    # Sum terms and get scalar part
    lagrangians = (momentum_term - mass_term - interaction_term)[""]

    # Sum lagrangians (one lagrangian for each spacetime point) over spacetime
    # to get a single value, the action.
    return tf.reduce_sum(lagrangians.blade_values)

action = get_action(psi=psi, a=a, spacing=0.1, electron_mass=1.0, electron_charge=1.0, hbar=1.0)
print("Action:", action)

Action: tf.Tensor(-280000.0, shape=(), dtype=float32)


Now that we can calculate the action for a given field configuration (ie. values for `psi` and `a` at every grid point) we could use a sampling algorithm
to sample fields and calculate quantities of interest such as the correlation function, vacuum energy and more.